In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web 
import plotly.graph_objects as go 


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [4]:
# Creating stock dataframe 
stock = web.DataReader('AAPL', data_source='yahoo', start = '2009-05-03', end = '2019-05-03')
stock = stock.reset_index()
stock

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-05-04,18.892857,18.240000,18.320000,18.867144,152339600.0,16.333237
1,2009-05-05,18.980000,18.731428,18.821428,18.958570,99563800.0,16.412386
2,2009-05-06,19.071428,18.602858,19.047142,18.928572,118384700.0,16.386417
3,2009-05-07,18.912857,18.271429,18.904285,18.437143,132944000.0,15.960989
4,2009-05-08,18.747143,18.037144,18.434286,18.455715,116991000.0,15.977063
...,...,...,...,...,...,...,...
2513,2019-04-29,205.970001,203.860001,204.399994,204.610001,22204700.0,201.421219
2514,2019-04-30,203.399994,199.110001,203.059998,200.669998,46534900.0,197.542618
2515,2019-05-01,215.309998,209.229996,209.880005,210.520004,64827300.0,207.239120
2516,2019-05-02,212.649994,208.130005,209.839996,209.149994,31996300.0,205.890457


In [67]:
# visualizing AAPL over the years
trace = go.Scatter(x = stock['Date'], y = stock['Close'], marker = {'color':"#3f8fcc"}, opacity=0.6)
layout = go.Layout(title = {'text':'AAPL 10Y <b>Close price</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}},plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace], layout = layout)
fig.show()

In [9]:
# constructing simple moving 40 day average
sma30 = pd.DataFrame()
sma30['close'] = stock['Close'].rolling(window = 30).mean()

# constructing simple moving 100 day average
sma100 = pd.DataFrame()
sma100['close'] = stock['Close'].rolling(window = 100).mean()


In [13]:
# visualizing SMAs
close = go.Scatter(x = stock.Date, y = stock.Close, name = "AAPL close price", marker = {'color':"#3f8fcc"}, opacity=0.4)
sma_short = go.Scatter(x = stock.Date, y = sma30['close'], name = "SMA 30")
sma_long = go.Scatter(x = stock.Date, y = sma100['close'], name = "SMA 100")
layout = go.Layout(title = {'text':'AAPL 10Y <b>Visualizing short term moving averages</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}},plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [close, sma_short, sma_long], layout = layout)
fig.show()



In [26]:
# storing SMA40, SMA100, AAPL close history 
df = pd.DataFrame()
df['AAPL'] = stock.Close
df['SMA30'] = sma30.close
df['SMA100'] = sma100.close


In [27]:
# signal to buy / sell data
def signal(data):
  sigPriceBuy = []
  sigPriceSell = []
  can_buy = True 
  can_sell = False

  for i in range(len(data)):
    if data['SMA30'][i] > data['SMA100'][i]:
      if can_buy:
        sigPriceBuy.append(data['AAPL'][i])
        sigPriceSell.append(np.nan)
        can_buy, can_sell = False, True
      else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
    
    elif data['SMA30'][i] < data['SMA100'][i]:
      if can_sell:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(data['AAPL'][i])
        can_buy, can_sell = True, False
      else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)

    else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)

  return sigPriceBuy, sigPriceSell




In [37]:
# store data and signal
buy_price, sell_price = signal(df)
df['Buy Signal Price'] = buy_price
df['Sell Signal Price'] = sell_price

In [46]:
# Analyzing RSI buy-sell signals
trace = go.Scatter(x = stock['Date'], y = stock['Close'], opacity = 0.5, name = "AAPL Close Price", marker = {'color':'#3f8fcc'})
buy = go.Scatter(x = stock.Date, y = df['Buy Signal Price'], marker = {'color':'green', 'symbol':'triangle-up','size':7}, name = "Buy", mode='markers')
sell = go.Scatter(x = stock.Date, y = df['Sell Signal Price'], marker = {'color':'red', 'symbol':'triangle-down','size':7}, name = "Sell", mode = 'markers')
layout = go.Layout(title = {'text':'AAPL 10Y <b>Buy-Sell Signals</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace,buy,sell], layout = layout)
fig.show()

In [59]:
# Accounting for profits and losses 
buy = [x for x in df['Buy Signal Price'] if str(x) != 'nan']
sell = [x for x in df['Sell Signal Price'] if str(x) != 'nan']
if len(buy) == len(sell) + 1:
  buy = buy[:-1]
profit = [] 
loss = []
for i in range(len(buy)):
  difference = sell[i] - buy[i]
  if  difference > 0:
    profit.append(difference)
  elif difference < 0:
    loss.append(difference)


In [68]:
# evaluating the profit 
adv = np.array(sell).copy().sum() - np.array(buy).copy().sum()
print("AAPL stats : ")
print("----------------------------------------------------------------------")
print("Min : " + str(stock['Close'].min()))
print("Max : " + str(stock['Close'].max()))
print("Trading time : 10Y (June 03 2009  - June 03 2019) " )
print("----------------------------------------------------------------------")
print('Total Profit using Dual Moving Avergage Crossover stratergy :' + str(adv))

AAPL stats : 
----------------------------------------------------------------------
Min : 17.06999969482422
Max : 232.07000732421875
Trading time : 10Y (June 03 2009  - June 03 2019) 
----------------------------------------------------------------------
Total Profit using Dual Moving Avergage Crossover stratergy :89.44001770019531


In [61]:
# Analysis of RSI trading stratergy
fig = go.Figure(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'green'}},
    value = np.array(sell).mean(),
    domain = {'x': [0.20, 1], 'y': [0.08, 0.25]},
    title = {'text': "Average value per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'red'}},
    value =  np.array(buy).mean(),
    domain = {'x': [0.20, 1], 'y': [0.4, 0.6]},
    title = {'text': "Average cost per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",
             'bar':{'color':'orange'}},
    delta = {'reference': stock['Close'][len(stock) - 1] - adv},
    value = stock['Close'][len(stock) - 1],
    domain = {'x': [0.20, 1], 'y': [0.7, 0.9]},
    title = {'text': "Current value"}))

fig.show()



In [62]:
# Use `hole` to create a donut-like pie chart
labels = ['Profit','Loss']
values = []
fig = go.Figure(data=[go.Pie(labels=labels, values=[len(profit),len(loss)], hole=.3,title={'text':"Share of P/L in all trades",})])
fig.show()


In [70]:
# Visualizing using a box plot

trace_buy = go.Box(y = profit, name = "Profit")
trace_sell = go.Box(y = np.array(loss)*-1, name = "Loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':1, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace_buy, trace_sell], layout = layout)
fig.show()

In [69]:
#using bubble plot 
x_profit = [i for i in range(len(profit))]
x_loss = np.array([i for i in range(len(profit), len(profit)+len(loss))])
trace1 = go.Scatter(x = x_profit, y = np.array(profit), marker={'size':np.array(profit) * 5}, mode="markers", name="profit")
trace2 = go.Scatter(x = x_loss, y = np.array(loss)*-1, marker={'size':np.array(loss)* -5}, mode="markers", name="loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':0.92, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace1, trace2], layout = layout)
fig.show()
